In [16]:
%pip install -qU duckdb psycopg2 jupysql pandas matplotlib duckdb-engine pandas-gbq

You should consider upgrading via the '/data/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Closing duckdb

In [1]:
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

In [2]:
%sql INSTALL postgres
%sql LOAD postgres

Running query in 'duckdb'

Running query in 'duckdb'

Success


In [3]:
# -- attach the Postgres database using the given connection string
%sql ATTACH 'host=localhost port=5432 dbname=worldcal user=worldcal_sa password=worldcalsa' AS production (TYPE postgres);
%sql USE production;

Running query in 'duckdb'

Running query in 'duckdb'

Success


In [4]:
%%sql
SELECT * FROM production.public.calendar_index
LIMIT 10;

Running query in 'duckdb'

id,title,summary,url_id,context,day,month,year,date_added
1,Start of the SpaceX fairing recovery program,SpaceX began an experimental program to determine if it might be possible to economically recover and reuse expended launch vehicle payload fairings from suborbital space.,49,{},0,0,2017,2024-04-20 19:36:33.932204
2,SpaceX begins routinely recovering fairings from many flights,"The experimental program became an operational program as, by late 2020, the company was routinely recovering fairings from many flights.",49,{},0,0,2020,2024-04-20 19:36:33.932204
3,SpaceX successfully refurbishes and reflies previously flown fairings on the majority of their satellite launches,"By 2021, SpaceX were successfully refurbishing and reflying previously flown fairings on the majority of their satellite launches.",49,{},0,0,2021,2024-04-20 19:36:33.932204
4,SpaceX begins chartering Ms. Tree and Ms. Chief,"SpaceX chartered two former platform supply vessels, Ms. Tree and Ms. Chief, to use as experimental platforms for recovery of rocket fairings from Falcon 9 orbital launch trajectories.",49,{},0,0,2018,2024-04-20 19:36:38.731961
5,Ms. Tree successfully catches a fairing,"Ms. Tree, while still named Mr. Steven, successfully caught a fairing during Falcon Heavy launch 3. This was the first successful fairing catch by SpaceX.",49,{},25,5,2019,2024-04-20 19:36:38.731961
6,Fairing recovery experiments,SpaceX conducted fairing recovery experiments on a number of occasions in 2018 and early 2019.,49,{},0,0,2018,2024-04-20 19:36:46.600274
7,First successful fairing catch,"Ms. Tree successfully caught a fairing during Falcon Heavy launch 3, which carried the DoD's STP-2 mission. This was the ship's first fairing recovery voyage after its renaming, change of ownership, and net upgrade.",49,{},25,6,2019,2024-04-20 19:36:46.600274
8,Operational status reached,"The fairing recovery program reached operational status where fairings from most Falcon 9 satellite launches were recovered, either 'in the net' or from the water.",49,{},0,0,2020,2024-04-20 19:36:46.600274
9,Final fairing caught in a net,The final fairing that was successfully caught in a net was in October 2020.,49,{},0,10,2020,2024-04-20 19:36:46.600274
10,Nets removed from fast ships,"In early 2021, the nets were removed from the two fast ships and SpaceX ended the ship leases, with both ships returned to their owner.",49,{},0,0,2021,2024-04-20 19:36:46.600274


In [11]:
%sql COPY production.public.spider_index TO '/workspace/worldcal/calendar_index/datasets/spider_index.csv' WITH (FORMAT CSV, HEADER);
%sql COPY production.public.calendar_index TO '/workspace/worldcal/calendar_index/datasets/calendar_index.csv' WITH (FORMAT CSV, HEADER);

Running query in 'duckdb'

Running query in 'duckdb'

Count
34134


In [7]:
import pandas as pd

df = pd.read_csv('/workspace/worldcal/calendar_index/datasets/calendar_index.csv')

In [ ]:
## New df without date_added
df = df.drop(columns=['date_added'])
df.head()

In [15]:
df.to_csv('/workspace/worldcal/calendar_index/datasets/calendar_index.csv', index=False)

In [8]:
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "the-world-calendar"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'calendar_index.calendar_events'
pandas_gbq.to_gbq(df, table_id, project_id=project_id)